In [1]:
import numpy as np
from numpy import identity as In
from numpy import zeros as Z
from numpy import ones as O
from numpy import matmul as M
from numpy import transpose as T
import numpy.linalg as lin
from numpy.linalg import inv as I

In [2]:
data=np.loadtxt("test.txt")
x=data[:,0:13]
y=data[:,13]
# this step is for standardizing the data 
#x=np.divide(np.subtract(x, np.mean(x, axis=0)), np.std(x, axis=0))
ones=np.ones((506,1))
x=np.append(ones, x, axis=1)

In [3]:
x_tran_x=M(T(x),x)
A=np.zeros((3,14))
A[0,6]=1;
A[1,7]=1;
A[2,13]=1
X = M(A,I(x_tran_x))
cov = I(M(X,T(A)))
cov

array([[   180.7708271 ,    157.28575607,   -782.8879059 ],
       [   157.28575607, 146172.46399901,  12293.94544685],
       [  -782.8879059 ,  12293.94544685,  13298.24618271]])

In [4]:
P=M(x, M( I(x_tran_x), T(x)))
P.shape

(506, 506)

In [5]:
#normalized data
beta_mle=M(M(I(x_tran_x),T(x)),y)
beta_mle

array([ 3.64594884e+01, -1.08011358e-01,  4.64204584e-02,  2.05586264e-02,
        2.68673382e+00, -1.77666112e+01,  3.80986521e+00,  6.92224640e-04,
       -1.47556685e+00,  3.06049479e-01, -1.23345939e-02, -9.52747232e-01,
        9.31168327e-03, -5.24758378e-01])

In [6]:
#non Normalized
beta_mle=M(M(I(x_tran_x),T(x)),y)
beta_mle

array([ 3.64594884e+01, -1.08011358e-01,  4.64204584e-02,  2.05586264e-02,
        2.68673382e+00, -1.77666112e+01,  3.80986521e+00,  6.92224640e-04,
       -1.47556685e+00,  3.06049479e-01, -1.23345939e-02, -9.52747232e-01,
        9.31168327e-03, -5.24758378e-01])

In [7]:
RSS=M(T(y), M((In(P.shape[0]) - P), y))
RSS
#M(A,beta_mle)

11078.784577973092

In [8]:

F_critical=3.86042832
for j in range(1,14):
    A=np.zeros((1,14))
    A[0,j]=1
    A_b=M(A, beta_mle)
    A_A=I(M(M(A, I(x_tran_x)),T(A)))
    RSS_h_minus_RSS= M(T(A_b), M(A_A, A_b))
    F=((RSS_h_minus_RSS)/RSS)*((506-14)/1)
    print(F)
    if(F>F_critical):
        print (j)

# for all these columns our hypothesis was not to include them in the model hence our hypothesis was worng 
# and wee need to include them.


10.801193141189264
1
11.435058151654273
2
0.11176320429826783
9.724299175093277
4
21.63419550605254
5
83.10401214426098
6
0.0027460143892066276
54.73045730667293
8
21.27884426548893
9
10.758459961109944
10
53.03496011394025
11
12.018650641052247
12
107.06342620906702
13


In [9]:
F_critical=3.86042832
# now are hypothesis is to include only these predictors
A=np.zeros((1,14))
A[0,3]=1
A[0,7]=1
A_b=M(A, beta_mle)
A_A=I(M(M(A, I(x_tran_x)),T(A)))
RSS_h_minus_RSS= M(T(A_b), M(A_A, A_b))
F=((RSS_h_minus_RSS)/RSS)*((506-14)/1)
F
# hence these are the significant predictors.


0.11420076194350136

In [10]:
t_alpha= 1.964797

In [11]:
a=np.zeros((14,1))
a[5,0]=1
denom=float((M(T(a), M(I(x_tran_x),a))*RSS/(506-14))**0.5)
upper=float(M(T(a),beta_mle)+t_alpha*denom)
lower=float(M(T(a),beta_mle)-t_alpha*denom)
print(" Predicted value of NOX",beta_mle[5], " Confidence Interval", "[ ",upper ,", ",lower, "]")

 Predicted value of NOX -17.76661122828687  Confidence Interval [  -10.26159025120765 ,  -25.27163220536609 ]


In [12]:
#now NOX as the only predictor

In [13]:
x_nox=x[:,[0,5]]
beta_nox=M(M(I(M(T(x_nox), x_nox)), T(x_nox)),y)
beta_nox
P_nox=M(x_nox, M( I(M(T(x_nox), x_nox)), T(x_nox)))
RSS_nox=M(T(y), M((In(P_nox.shape[0]) - P_nox), y))
t_alpha_nox=1.964682
a=np.zeros((2,1))
a[1,0]=1
denom=float((M(T(a), M(I(M(T(x_nox), x_nox)),a))*RSS_nox/(506-2))**0.5)
upper=float(M(T(a),beta_nox)+t_alpha_nox*denom)
lower=float(M(T(a),beta_nox)-t_alpha_nox*denom)
print(" Predicted value of NOX",beta_nox[1], " Confidence Interval", "[ ",upper ,", ",lower, "]")

 Predicted value of NOX -33.91605500866066  Confidence Interval [  -27.636269175572966 ,  -40.195840841748364 ]


In [14]:
#F_critical =254.06128185

A=np.zeros((1,2))
A[0,1]=1
A_b=M(A, beta_nox)
A_A=I(M(M(A, I(M(T(x_nox), x_nox))),T(A)))
RSS_h_minus_RSS= M(T(A_b), M(A_A, A_b))
F=((RSS_h_minus_RSS)/RSS_nox)*((506-2)/1)
print(F)
#if(F>F_critical):
#        print (j)

112.59148027969351


In [15]:
A_b

array([-33.91605501])

In [16]:
x_last=x[:,[6,7,13]]
x_tran_x=M(T(x_last),x_last)
P=M(x_last, M( I(x_tran_x), T(x_last)))
beta_mle=M(M(I(x_tran_x),T(x_last)),y)
RSS=M(T(y), M((In(P.shape[0]) - P), y))
x_tran_x

array([[  20234.598247,  215670.1759  ,   38681.78802 ],
       [ 215670.1759  , 2779614.63    ,  500191.627   ],
       [  38681.78802 ,  500191.627   ,  106762.9583  ]])

In [17]:
beta_mle

array([ 4.85493231,  0.00938516, -0.68087926])

In [18]:
F_critical=2.62262805